# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [2]:
#load data from the given path
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X = adult_dt.drop(columns=['income'])
Y = adult_dt['income']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("Y_train shape:", Y_train.shape)
print("Y_test shape:", Y_test.shape)

X_train shape: (22792, 14)
X_test shape: (9769, 14)
Y_train shape: (22792,)
Y_test shape: (9769,)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

*(Comment here.)*

The "random state" parameter in the "train_test_split" function is an integer value that ensures the split's reproducibility. The random state is a seed for the random number generator that shuffles and splits data.
It is useful since we ensure that data is split the same way every time we run the code, by setting a specific value for "random state". Moreover, this allows for the consistency and comparability of results across different runs and experiments. Overall, the useful reasons of the function are reproducibility, debugging, comparison and collaboration.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, OneHotEncoder

In [9]:
num_features = adult_dt.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_features = adult_dt.select_dtypes(include=['object']).columns.tolist()

In [10]:
num_pipeline = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', RobustScaler())
])

In [11]:
cat_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
])

In [13]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, num_features),
        ('cat', cat_pipeline, cat_features)
    ]
)


In [14]:
preprocessed_data = preprocessor.fit_transform(adult_dt)

In [15]:
preprocessed_df = pd.DataFrame(preprocessed_data.toarray() if hasattr(preprocessed_data, "toarray") else preprocessed_data)
print(preprocessed_df.head())

    0         1         2       3    4    5    6    7    8    9    ...  97   \
0  0.10 -0.845803  1.000000  2174.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
1  0.65 -0.797197  1.000000     0.0  0.0 -5.4  0.0  0.0  0.0  0.0  ...  0.0   
2  0.05  0.312773 -0.333333     0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
3  0.80  0.472766 -1.000000     0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
4 -0.45  1.342456  1.000000     0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   

   98   99   100  101  102  103  104  105  106  
0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  
1  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  
2  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  
3  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  
4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 107 columns]


# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [16]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate

In [17]:
num_features = adult_dt.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_features = adult_dt.select_dtypes(include=['object']).columns.tolist()

In [18]:
num_pipeline = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', RobustScaler())
])

In [19]:
cat_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
])

In [20]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, num_features),
        ('cat', cat_pipeline, cat_features)
    ]
)

In [21]:
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])

In [22]:
scoring = ['neg_log_loss', 'roc_auc', 'accuracy', 'balanced_accuracy']


In [23]:
cv_results = cross_validate(model_pipeline, adult_dt.drop(columns=['income']), adult_dt['income'], 
                            cv=5, scoring=scoring, return_train_score=True)
cv_results_df = pd.DataFrame(cv_results)
print(cv_results_df)

c:\Users\Admin\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Admin\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/lin

   fit_time  score_time  test_neg_log_loss  train_neg_log_loss  test_roc_auc  \
0  3.012971    0.116359          -0.325234           -0.316496      0.903904   
1  1.617007    0.062547          -0.330222           -0.315080      0.899250   
2  2.489653    0.107346          -0.318156           -0.318062      0.907950   
3  2.607298    0.117455          -0.314492           -0.319003      0.910278   
4  2.619047    0.075272          -0.311184           -0.320021      0.911004   

   train_roc_auc  test_accuracy  train_accuracy  test_balanced_accuracy  \
0       0.908238       0.847382        0.853693                0.758507   
1       0.909600       0.848127        0.853315                0.761930   
2       0.907505       0.851966        0.851818                0.767071   
3       0.906949       0.857340        0.851319                0.773006   
4       0.906307       0.852580        0.851933                0.770306   

   train_balanced_accuracy  
0                 0.768481  
1         

Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [24]:
num_features = adult_dt.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_features = adult_dt.select_dtypes(include=['object']).columns.tolist()

In [25]:
num_pipeline = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', RobustScaler())
])
cat_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
])


In [26]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, num_features),
        ('cat', cat_pipeline, cat_features)
    ]
)

In [27]:
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])
scoring = ['neg_log_loss', 'roc_auc', 'accuracy', 'balanced_accuracy']

In [28]:
cv_results = cross_validate(model_pipeline, adult_dt.drop(columns=['income']), adult_dt['income'], 
                            cv=5, scoring=scoring, return_train_score=True)

c:\Users\Admin\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Admin\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/lin

In [29]:
cv_results_df = pd.DataFrame(cv_results)

In [30]:
sorted_cv_results_df = cv_results_df.sort_values(by='test_neg_log_loss', ascending=False)
print(sorted_cv_results_df)

   fit_time  score_time  test_neg_log_loss  train_neg_log_loss  test_roc_auc  \
4  2.606433    0.081595          -0.311184           -0.320021      0.911004   
3  2.568355    0.080705          -0.314492           -0.319003      0.910278   
2  2.768411    0.081352          -0.318156           -0.318062      0.907950   
0  1.784269    0.049908          -0.325234           -0.316496      0.903904   
1  1.625402    0.082406          -0.330222           -0.315080      0.899250   

   train_roc_auc  test_accuracy  train_accuracy  test_balanced_accuracy  \
4       0.906307       0.852580        0.851933                0.770306   
3       0.906949       0.857340        0.851319                0.773006   
2       0.907505       0.851966        0.851818                0.767071   
0       0.908238       0.847382        0.853693                0.758507   
1       0.909600       0.848127        0.853315                0.761930   

   train_balanced_accuracy  
4                 0.765662  
3         

Calculate the mean of each metric. 

In [32]:
mean_metrics = cv_results_df.mean()
print(mean_metrics)

fit_time                   2.270574
score_time                 0.075193
test_neg_log_loss         -0.319858
train_neg_log_loss        -0.317732
test_roc_auc               0.906477
train_roc_auc              0.907720
test_accuracy              0.851479
train_accuracy             0.852415
test_balanced_accuracy     0.766164
train_balanced_accuracy    0.767455
dtype: float64


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [36]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score, log_loss

In [33]:
X_train, X_test, Y_train, Y_test = train_test_split(adult_dt.drop(columns=['income']), adult_dt['income'], 
                                                    test_size=0.3, random_state=42)


In [34]:
model_pipeline.fit(X_train, Y_train)
Y_pred_proba = model_pipeline.predict_proba(X_test)
Y_pred = model_pipeline.predict(X_test)

c:\Users\Admin\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [37]:
metrics = {
    'neg_log_loss': -log_loss(Y_test, Y_pred_proba),
    'roc_auc': roc_auc_score(Y_test, Y_pred_proba[:, 1]),
    'accuracy': accuracy_score(Y_test, Y_pred),
    'balanced_accuracy': balanced_accuracy_score(Y_test, Y_pred)
}
print(metrics)

{'neg_log_loss': -0.31793986716589256, 'roc_auc': 0.9053125436572185, 'accuracy': 0.8548469648889344, 'balanced_accuracy': 0.7684056514251165}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)

Recoding the target variable income immediately upon loading the data is highly convenient for several reasons. First, it ensures data consistency by transforming the target variable into the desired format right from the start, which helps avoid potential errors or inconsistencies later in the workflow. This immediate transformation also simplifies the overall data preparation process, as it reduces the need for repetitive transformations or checks in different parts of the code, leading to a cleaner and more manageable workflow.
Moreover, having a binary target variable (e.g., 0 and 1) is particularly convenient for modeling, especially for classification tasks, as many machine learning algorithms expect numeric target variables. This early recording minimizes the risk of forgetting to transform the target variable before model training, ensuring that all necessary data preparation steps are completed upfront.
Additionally, recoding the target variable at the data loading stage enhances the readability of the code. It indicates that the income variable is being transformed and explains how this transformation is being done, making the code easier to understand and maintain. Overall, this approach ensures that the data is in the correct format from the beginning, facilitating smoother and more efficient subsequent steps in data analysis and modeling.

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Evaluation of model pipeline |Model pipeline was evaluated correctly.|Model pipeline was not evaluated correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.